In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from pandas.io import sql
import os
import pandas as pd
# from decouple import config
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from PIL import Image
from matplotlib import cm

from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized"); # chrome 브라우저를 최대한 크기로 옵션을 주는 것

# Chrome 실행
browser = webdriver.Chrome('./chromedriver', options=options)

In [2]:
def category_df(x):
    # bigfoo9 사이트
    browser.get("https://bigfoot9.com/youtube/today-channel")
    browser.implicitly_wait(5)

    # 카테고리 영역 클릭
    browser.find_element_by_xpath('//*[@id="container"]/div/section/div[1]/ul[1]/li[6]/div/i').click()
    
    if x == "뷰티":
        # 패션/뷰티 카테고리 클릭
        browser.find_element_by_xpath('//*[@id="category_list"]/li[33]/span[1]').click()
    elif x == "먹방":
        # 먹방 클릭
        browser.find_element_by_xpath('//*[@id="category_list"]/li[15]/span[1]').click()
    elif x == "여행":
        # 여행 클릭
        browser.find_element_by_xpath('//*[@id="category_list"]/li[20]/span[1]').click()
    elif x == "게임":
        # 게임 클릭
        browser.find_element_by_xpath('//*[@id="category_list"]/li[4]/span[1]').click()
    elif x == "정치":
        # 정치 클릭
        browser.find_element_by_xpath('//*[@id="category_list"]/li[28]/span[1]').click()

    # View수 클릭
    browser.find_element_by_xpath('//*[@id="page-rank"]/div[1]/ul/li[2]').click()
    # more 버튼 클릭
    browser.find_element_by_xpath('//*[@id="page-rank"]/div[2]/input').click()

    # 기준 url
    web_standard = browser.find_element_by_xpath('//*[@id="tablePages"]/tbody')

    # concat에 쓰일 빈 리스트 생성
    df_lst = []

    for idx in range(2, 22):
        df = pd.DataFrame({
            "category":[x],
            "id":[web_standard.find_element_by_xpath(f"tr[{idx}]").text.split("\n")[0]],
            "url":[web_standard.find_element_by_xpath(f"tr[{idx}]").text.split("\n")[1]],
            "subscribers":[web_standard.find_element_by_xpath(f"tr[{idx}]").text.split("\n")[5]]
        })
        df_lst.append(df)

    #  
    result_df = pd.concat(df_lst).reset_index(drop = True)
    return result_df

In [3]:
def thumnail_download(result_df, order):
    # 유튜버 페이지 이동
    browser.get("https://" + result_df["url"][order])

    # 동영상 탭 클릭
    browser.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div').click()

    # folder 생성
    if "/" in result_df["id"][order]:
        name = result_df["id"][order].replace("/", "_")
        if not os.path.isdir('../data/youtube/{}'.format(name)):
            os.mkdir('../data/youtube/{}'.format(name))
    else:
        if not os.path.isdir('../data/youtube/{}'.format(result_df["id"][order])):
            os.mkdir('../data/youtube/{}'.format(result_df["id"][order]))


    # 동영상 탭 url 추출
    base_url = browser.current_url
    # 인기동영상 정렬 url 추출
    popular_url = "?view=0&sort=p&flow=grid"
    # 인기동영상 정렬 이동
    browser.get(base_url + popular_url)

    # 스크롤 내리기
    body = browser.find_element_by_css_selector('body')
    for i in range(5):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)




    youtube_lst = []

    for idx in range(1, 41):
        df = pd.DataFrame({"category":[result_df["category"][order]],
            "id":[result_df["id"][order]],
            "subscribers":[result_df["subscribers"][order]],
            "video_name":[browser.find_element_by_xpath(f'//*[@id="items"]/ytd-grid-video-renderer[{str(idx)}]').text.split("\n")[1]],
            "video_watch":[browser.find_element_by_xpath(f'//*[@id="items"]/ytd-grid-video-renderer[{str(idx)}]').text.split("\n")[2]]
        })

        # 이미지 파일
        # xpath
        img_xpath = f'/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[{idx}]/div[1]/ytd-thumbnail/a/yt-img-shadow/img'

        image = browser.find_element_by_xpath(img_xpath)
        # url 추출
        img_url = image.get_attribute('src')
        id = img_url.replace("https://i.ytimg.com/vi/", "")
        url = f"http://i.ytimg.com/vi/{id}/maxresdefault.jpg"
        if "/" in result_df["id"][order]:
            name = result_df["id"][order].replace("/", "_")
            urlretrieve(url, f"../data/youtube/{name}/{str(idx)}.jpg")
            img = Image.open(f"../data/youtube/{name}/{str(idx)}.jpg")
        else:
            urlretrieve(url, f"../data/youtube/{result_df['id'][order]}/{str(idx)}.jpg")
            img = Image.open(f"../data/youtube/{result_df['id'][order]}/{str(idx)}.jpg")
        df["image"] = img
        youtube_lst.append(df)

    df_final = pd.concat(youtube_lst)
    return df_final

In [4]:
lst = []
category_lst = ["먹방", "여행", "게임", "정치"]

try:
    for x in category_lst:
        result_df = category_df(x)
        for idx in range(len(result_df)):
            print(idx)
            time.sleep(2)
            a = thumnail_download(result_df, idx)
            lst.append(a)
except Exception as e:
    print(e)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [39]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 799
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     800 non-null    object
 1   id           800 non-null    object
 2   subscribers  800 non-null    object
 3   video_name   800 non-null    object
 4   video_watch  800 non-null    object
 5   image        800 non-null    object
dtypes: object(6)
memory usage: 43.8+ KB


In [5]:
a = pd.concat(lst).reset_index(drop = True)
# a["image_ndarray"] = a["image"].apply(lambda x: np.array(x))

def watch_to_int(x):
    a = x.replace("조회수", "")
    a = a.replace(" ", "")
    a = a.replace("만회", "0000")
    a = a.replace("억회", "000000000")
    a = a.replace(".", "")
    a = int(a)
    return a

a["video_watch"] = a["video_watch"].apply(lambda x: watch_to_int(x))

def subs_to_int(x):
    a = x.replace(",", "")
    a = int(a)
    return a

a["subscribers"] = a["subscribers"].apply(lambda x: subs_to_int(x))
a["watch/sub"] = a["video_watch"] / a["subscribers"]

In [9]:
a.head(3)

,category,id,subscribers,video_name,video_watch,image,watch/sub
0,먹방,Jane ASMR 제인,10900000,"ASMR RAINBOW CREPE CAKE, EDIBLE SPOON, MACARON...",17000000000,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1559.633028
1,먹방,Jane ASMR 제인,10900000,ASMR BLACK BEAN NOODLES + FRIED CHICKEN 시크릿 양념...,92530000,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8.488991
2,먹방,Jane ASMR 제인,10900000,"ASMR EDIBLE HAIR BRUSH, SPOON, BUBBLE TEA, GUM...",89080000,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8.172477


In [28]:
# image save

for idx in range(len(a)):
    im = a["image"][idx]
    im.save(f"/Users/mac/Desktop/bigdata/Python/data/{str(idx)}.jpg")

In [14]:
import pickle
my_data = {"category":a["category"],
           "id":a["id"],
           "subscribers": a["subscribers"],
           "video_name":a["video_name"],
           "vide_watch":a["video_watch"],
           "image":a["image"],
          "watch/sub":a["watch/sub"]}
output = open('youtube_thumnail.pkl', 'wb')
pickle.dump(my_data, output)
output.close()

pkl_file = open('youtube_thumnail.pkl', 'rb')

data1 = pickle.load(pkl_file)

pkl_file.close()

In [29]:
data1

{'category': 0       먹방
 1       먹방
 2       먹방
 3       먹방
 4       먹방
         ..
 3195    정치
 3196    정치
 3197    정치
 3198    정치
 3199    정치
 Name: category, Length: 3200, dtype: object,
 'id': 0          Jane ASMR 제인
 1          Jane ASMR 제인
 2          Jane ASMR 제인
 3          Jane ASMR 제인
 4          Jane ASMR 제인
              ...       
 3195    SesameTube 참깨방송
 3196    SesameTube 참깨방송
 3197    SesameTube 참깨방송
 3198    SesameTube 참깨방송
 3199    SesameTube 참깨방송
 Name: id, Length: 3200, dtype: object,
 'subscribers': 0       10900000
 1       10900000
 2       10900000
 3       10900000
 4       10900000
           ...   
 3195      148000
 3196      148000
 3197      148000
 3198      148000
 3199      148000
 Name: subscribers, Length: 3200, dtype: int64,
 'video_name': 0       ASMR RAINBOW CREPE CAKE, EDIBLE SPOON, MACARON...
 1       ASMR BLACK BEAN NOODLES + FRIED CHICKEN 시크릿 양념...
 2       ASMR EDIBLE HAIR BRUSH, SPOON, BUBBLE TEA, GUM...
 3       ASMR JELLY NOODLES, GUMMY JE